# Background 

JSON files are often used not just to store data, but also to store configuration information for how a program should run. This way, your program doesn't have to be changed when minor configuration changes are all that need to change. Examples might include database configuration information, source database table names, date ranges for data processing, etc. The final includes a JSON file that provides important configuration information that will be used to direct the rest of your program on how to combine and process the data from an XML file and a SQL database. First, however, you'll need to read all the instructions and describe your approach / algorithm for solving the final exam.
JSON Configuration --------------------------------------\
                                                          >---> Total Cost per Claim
SQL Database: Prescription Claims ---------------\       /
                                                  >-----/
XML File: Mapping of Member Group to Admin Fee --/



#Step 1
#Write a function called setup to Read /final/config_sample.json and return a dictionary 
#that has key/value pairs,member group as the key and a list of cost field names as the values

#Step 2
#Write a function called read_fees that will read /final/fees_sample.xml and return the children of the parent root
#in a dictionary format, where the key is member group, with subkeys :mail/retails, and sub-sub 
#keys type of fee and value cost. The function should do this for every child of parent root,fee_config


#Step 3
#Write a function called calc_total_cost that takes parameters:(
#config, the disctionary returned from the setup function
#fees, the dictionary returned from the read_fees function
#member_group
#mail_retail
#) and returns the resulting total cost.

#Step 4
#Write function to read data from SQL database, that utilizes previous functions to calculate total cost and
#return total cost as an attribute in th database

#Step 4a
Pull all fields from the claims database by using the Setup module to identify important login information

Step 4b
#Match the member group ID and the mail_retail code from the claims filed for the prices needed in the fees. Take the #ingredient cost the the claims database. Sum all the cost together and round to the 2 dec places

#Step 4c
Update the the claims database with new key, Total cost, and result from the sum process

#Step 5
Check to make sure cost is accurate and dedug if neccessary

In [1]:
import xml.etree.ElementTree as xml
import json
import doctest
import pymysql.cursors
import jupyterimporter
import Setup 
import read_fees
import calc_total_cost

importing Jupyter notebook from Setup.ipynb
importing Jupyter notebook from read_fees.ipynb
importing Jupyter notebook from calc_total_cost.ipynb


In [2]:
filename="/final/config_sample.json"
filename_xml="/final/fees_sample.xml"

In [3]:
'''This function turns the json file into dict format'''
Setup.setup_two(filename)

{'Cost Field Mapping': {'AZ001': ['incredient_cost', 'admin_fee'],
  'MG001': ['ingredient_cost', 'admin_fee', 'rebate_amount'],
  'MG002': ['ingredient_cost']},
 'Database Host': 'localhost',
 'Database Name': 'hds5210',
 'Database Password': 'S1UC0R',
 'Database User': 'slucor',
 'End Date': '2016-11-30',
 'Start Date': '2016-11-01'}

In [4]:
'''This fnx extract from the above dict and outputs only my 
cost related information'''
Setup.setup(filename)

{'AZ001': ['incredient_cost', 'admin_fee'],
 'MG001': ['ingredient_cost', 'admin_fee', 'rebate_amount'],
 'MG002': ['ingredient_cost']}

In [5]:
'''This fnx creates a map between the memberid group, retail_
mail code and some of the cost. This fnx depend on the availability of
others(extract, find_unique_membergroup etc see attached)'''
read_fees.read_fees(filename_xml)

importing Jupyter notebook from find_unique_membergroup.ipynb
importing Jupyter notebook from xml_to_dict.ipynb
importing Jupyter notebook from listToKey_Val.ipynb
importing Jupyter notebook from extract.ipynb


{'MG001': {'MAIL': {'admin_fee': '0.50', 'processing_fee': '0.05'},
  'RETAIL': {'admin_fee': '0.50', 'processing_fee': '0.10'}}}

In [6]:
d=Setup.setup('/final/config.json')
c=Setup.setup_two('/final/config.json')
f=read_fees.read_fees('/final/fees.xml')
connection = pymysql.connect(
    host=c.get('Database Host'),
    user=c.get('Database User'),
    password=c.get('Database Password'),
    db=c.get('Database Name'),
    cursorclass=pymysql.cursors.DictCursor)

cursor = connection.cursor()
sqlList='SELECT * FROM claims'
cursor.execute(sqlList)
result=cursor.fetchall()
connection.close()

In [7]:
'''This fnx pull the member group id of interest, the mail_order code
then uses the calc_total_cost func along with in igredient cost 
from the claims file to calclulate a TOTAL COST'''
config=d
fees=f
for i in range(len(result)):
    ingredient_cost=result[i].get('ingredient_cost')
    member_group=result[i].get('member_group')
    mail_retail=result[i].get('mail_order_ind')
    result[i].update({'TOTAL COST':calc_total_cost.calc_total_cost(config,fees,member_group,mail_retail,ingredient_cost)}) 

In [8]:
result[901]

{'TOTAL COST': 4.6,
 'benefit_plan_code': 'PBM',
 'brand_generic_ind': 'GENERIC',
 'business_unit_id': 'HNCA',
 'copay_amount': Decimal('3.66000'),
 'drug_ndc': '591533501',
 'ingredient_cost': Decimal('2.41000'),
 'lob_code': '',
 'lob_desc': '',
 'mail_order_ind': 'RETAIL',
 'member_group': '2005',
 'new_refill_indicator': 'REFILL',
 'pa_indicator': 'N',
 'paid_amount': Decimal('0.00000'),
 'process_date': '2015-01-05',
 'rx_count': '',
 'rx_number': '913206',
 'sales_tax': Decimal('0.00000')}

In [9]:
config['2005']

['ingredient_cost', 'admin_fee', 'processing_fee']

In [10]:
fees['2005']

{'MAIL': {'admin_fee': '0.07', 'processing_fee': '0.31'},
 'RETAIL': {'admin_fee': '0.83', 'processing_fee': '1.36'}}

In [ ]:
%%bash
cd ~/notebooks/hds5210-week02/
git add Final-williams0692.ipynb
git add calc_total_cost.ipynb
git add extract.ipynb
git add find_unique membergroup.ipynb
git add read_fees.ipynb
git add Setup.ipynb
git add xml_to_dict.ipynb
git add listToKey_Val.ipynb
git commit -a -m "Adding Final"
git push